## Loading the dataset

In [6]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('Train:', len(x_train))
print('Test:', len(x_test))

Train: 50000
Test: 10000


## Visualising the first 36 images 

In [7]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(20,5))
for i in range(36):
    ax = fig.add_subplot(3, 12, i+1, xticks=[], yticks=[])
    ax.imshow(x_train[i])

## Scale the values 

In [8]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

## Splitting the train set in: train set and validation set and one-hot encoding 

In [9]:
from keras.utils import np_utils

num_classes = len(np.unique(y_train))
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

print('Train samples:', len(x_train))
print('Validation samples:', len(y_valid))

Train samples: 45000
Validation samples: 5000


## Defining the model 

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(32, 32, 3)))

model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))

model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))

model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
__________

## Compiling the model 

In [3]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

## Training the model  

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='w_best_CNN.hdf5', save_best_only=True, verbose=1)

model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_valid, y_valid), callbacks=[checkpointer], verbose=2, shuffle=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/30
 - 457s - loss: 1.5483 - acc: 0.4454 - val_loss: 1.3908 - val_acc: 0.5206

Epoch 00001: val_loss improved from inf to 1.39079, saving model to w_best_CNN.hdf5
Epoch 2/30
 - 442s - loss: 1.1190 - acc: 0.6116 - val_loss: 1.1304 - val_acc: 0.6004

Epoch 00002: val_loss improved from 1.39079 to 1.13042, saving model to w_best_CNN.hdf5
Epoch 3/30
 - 410s - loss: 0.9677 - acc: 0.6690 - val_loss: 0.9832 - val_acc: 0.6602

Epoch 00003: val_loss improved from 1.13042 to 0.98318, saving model to w_best_CNN.hdf5
Epoch 4/30
 - 412s - loss: 0.9169 - acc: 0.6874 - val_loss: 1.1011 - val_acc: 0.6494

Epoch 00004: val_loss did not improve from 0.98318
Epoch 5/30
 - 410s - loss: 0.9057 - acc: 0.6994 - val_loss: 0.8923 - val_acc: 0.7116

Epoch 00005: val_loss improved from 0.98318 to 0.89234, saving model to w_best_CNN.hdf5
Epoch 6/30
 - 410s - loss: 0.9148 - acc: 0.7004 - val_loss: 0.9730 - val_acc: 0.6998

Epoch 00006: val_loss did not improv

In [10]:
model.load_weights('w_best_CNN.hdf5')

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
print(score[1])

0.7047
